# **Build Agent**

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import logging
from langchain.utilities import SQLDatabase

from src.database import create_sql_engine
from src.schema import create_schema
from src.llm import init_llm_gemini
from src.retriever import init_retriever
from agent import build_agent

GOOGLE_API = os.getenv("GOOGLE_API")

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s | %(message)s')

In [2]:
# Initialize SQLite DB
engine, _ = create_sql_engine()
db = SQLDatabase(engine, include_tables=["members", "items", "campaigns", "transactions", "transaction_items"])

INFO | Database schema created at sqlite:///:memory:.
INFO | Generating 100 members...
INFO | Generating 30 items...
INFO | Generating 5 campaigns...
INFO | Generating 150 transactions...
INFO | All generated data added to session and committed.


In [3]:
# Initialize schema retriever and tools
llm = init_llm_gemini(api_key=GOOGLE_API)
df_schema = create_schema()
retriever = init_retriever(df_schema)

INFO | Initializing Online LLM with model: models/gemini-2.0-flash...
INFO | Initializing Chroma retriever with embedding model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
INFO | Use pytorch device_name: mps
INFO | Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
INFO | Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
ERROR | Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR | Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [4]:
# Build agent
agent = build_agent(llm, db, retriever, verbose=False)

/Users/lanlan/Desktop/Text2SQL-Agent/agent.py:113: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sql_generation_chain = LLMChain(llm=llm, prompt=chain_prompt, verbose=verbose)
/Users/lanlan/Desktop/Text2SQL-Agent/agent.py:210: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = ZeroShotAgent(


In [5]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:


list_tables(tool_input: 'str', callbacks: 'Callbacks' = None) -> 'str' - Use this tool to list all available table names in the database.
describe_table(table) - Use this tool to describe the schema of a specific table, including column names and data types.
sql_query(question, max_retries=3) - Use this tool to answer questions about user data, metrics, or reports from the database. Input should be a complete question in natural language. The tool will automatically generate, execute, and correct SQL to find the answer.
schema_lookup(query) - A tool to retrieve definitions of table or column names. Use when the input is a natural language question containing a field or table name that needs clarification.Input should be a short query or phrase asking about the meaning or definition of a table or column. Returns the associated schema documentation.

When answering questions, follow this structured 

# **Run Test Cases**

In [6]:
import time

## *Trial E1*

In [7]:
print(agent.invoke("Show the names and categories of all items with a rating above 3.0."))
time.sleep(30)

INFO | Attempt 1 of 3 in query db data
INFO | Executing SQL: SELECT item_name, category FROM items WHERE rating > 3.0


{'input': 'Show the names and categories of all items with a rating above 3.0.', 'output': "[('Stuff Guess', 'Sports & Outdoors'), ('Size Between', 'Sports & Outdoors'), ('Everyone Product', 'Sports & Outdoors'), ('Someone Energy', 'Sports & Outdoors'), ('Room Born', 'Home & Kitchen'), ('Investment Avoid', 'Beauty & Personal Care'), ('As Save', 'Apparel'), ('Check Memory', 'Electronics'), ('Because Benefit', 'Electronics'), ('Serious Space', 'Beauty & Personal Care'), ('Accept Policy', 'Sports & Outdoors'), ('Single Yes', 'Beauty & Personal Care'), ('Too Technology', 'Home & Kitchen'), ('Government Send', 'Apparel'), ('Matter Again', 'Electronics'), ('Admit Development', 'Electronics'), ('Pull Employee', 'Apparel')]"}


## *Trial E2*

In [8]:
print(agent.invoke("How many transactions were made using LinePay?"))
time.sleep(30)

INFO | Attempt 1 of 3 in query db data
INFO | Executing SQL: SELECT count(*) FROM transactions WHERE payment_method = 'LinePay'


{'input': 'How many transactions were made using LinePay?', 'output': '36'}


## *Trial E3*

In [9]:
print(agent.invoke("What are the names of members who made a purchase during the campaign with campaign_id=1?"))
time.sleep(30)

INFO | Attempt 1 of 3 in query db data
INFO | Executing SQL: SELECT DISTINCT T1.member_name FROM members AS T1 INNER JOIN transactions AS T2 ON T1.member_id = T2.member_id WHERE T2.campaign_id = 1


{'input': 'What are the names of members who made a purchase during the campaign with campaign_id=1?', 'output': 'John Knox, Colleen Stanton, Benjamin Mcdaniel, Robert Campos'}


## *Trial E4*

In [10]:
print(agent.invoke("Find the total final price of all transactions made by members from the 'USA'."))
time.sleep(30)

INFO | Attempt 1 of 3 in query db data
INFO | Executing SQL: SELECT SUM(T1.final_price) FROM transactions AS T1 INNER JOIN members AS T2 ON T1.member_id = T2.member_id WHERE T2.country = 'USA'


{'input': "Find the total final price of all transactions made by members from the 'USA'.", 'output': '176893.55000000002'}


## *Trial E5*

In [11]:
print(agent.invoke("For each item category, what is the name and price of the most expensive item? Show the category, item name, and price."))
time.sleep(30)

INFO | Attempt 1 of 3 in query db data
INFO | Executing SQL: SELECT
  category,
  item_name,
  price
FROM (
  SELECT
    category,
    item_name,
    price,
    RANK() OVER (PARTITION BY category ORDER BY price DESC) AS price_rank
  FROM items
)
WHERE
  price_rank = 1;


SQLResult:

category	item_name	price
Beauty & Health	Few Stuff	1999.44
Electronics	Kind Voice	1999.17
Home & Kitchen	Trouble Idea	1996.47
Sports & Outdoors	Pop Sure	1998.74
Toys & Games	Hard Region	1997.91


Answer:
The most expensive item in the Beauty & Health category is 'Few Stuff' with a price of 1999.44. The most expensive item in the Electronics category is 'Kind Voice' with a price of 1999.17. The most expensive item in the Home & Kitchen category is 'Trouble Idea' with a price of 1996.47. The most expensive item in the Sports & Outdoors category is 'Pop Sure' with a price of 1998.74. The most expensive item in the Toys & Games category is 'Hard Region' with a price of 1997.91.
INFO | Attempt 2 of 3 in query db

{'input': 'For each item category, what is the name and price of the most expensive item? Show the category, item name, and price.', 'output': 'Apparel: As Save, 1714.57; Beauty & Personal Care: Single Yes, 1235.54; Electronics: Because Benefit, 1848.64; Home & Kitchen: Race Senior, 884.9; Sports & Outdoors: Under Behavior, 1885.32'}


## *Trial C1*

In [12]:
print(agent.invoke("列出所有在 2024 年後加入，且會員等級為 'Bronze' 的會員姓名與電子郵件，並按照加入日期從新到舊排序。"))
time.sleep(30)

INFO | Attempt 1 of 3 in query db data
INFO | Executing SQL: SELECT member_name, email FROM members WHERE STRFTIME('%Y', join_date) > '2024' AND member_level = 'Bronze' ORDER BY join_date DESC


{'input': "列出所有在 2024 年後加入，且會員等級為 'Bronze' 的會員姓名與電子郵件，並按照加入日期從新到舊排序。", 'output': 'Anthony Moreno (hollyclark@example.org), Christopher Thomas (fbaker@example.org), Jennifer Rivera (jordanbowen@example.net), Colleen Wilson (eddiemiller@example.net), Daniel Fitzgerald (andrew98@example.com), Allison Harris (olittle@example.com), Samantha Bell (iking@example.net)'}


## *Trial C2*

In [13]:
print(agent.invoke("campaign_id=3 這個行銷活動總共產生了幾筆交易？"))
time.sleep(30)

INFO | Attempt 1 of 3 in query db data
INFO | Executing SQL: SELECT count(*) FROM transactions WHERE campaign_id = 3


{'input': 'campaign_id=3 這個行銷活動總共產生了幾筆交易？', 'output': '6'}


## *Trial C3*

In [14]:
print(agent.invoke("哪個品牌的商品總庫存量最多？請顯示品牌名稱與總庫存量。"))
time.sleep(30)

INFO | Attempt 1 of 3 in query db data
INFO | Executing SQL: SELECT brand, sum(stock_quantity) AS total_stock_quantity FROM items GROUP BY brand ORDER BY total_stock_quantity DESC LIMIT 1


{'input': '哪個品牌的商品總庫存量最多？請顯示品牌名稱與總庫存量。', 'output': 'Adidas品牌的商品總庫存量最多，總庫存量為1132。'}


## *Trial C4*

In [15]:
print(agent.invoke("找出所有由 member_id=5 推薦加入的會員，他們總共創造了多少成交價格？"))
time.sleep(30)

INFO | Attempt 1 of 3 in query db data
INFO | Executing SQL: SELECT SUM(T1.final_price) FROM transactions AS T1 INNER JOIN members AS T2 ON T1.member_id = T2.member_id WHERE T2.referrer_id = 5


{'input': '找出所有由 member_id=5 推薦加入的會員，他們總共創造了多少成交價格？', 'output': '2123.32'}


## *Trial C5*

In [16]:
print(agent.invoke("所有交易紀錄中，單筆交易購買了最多品項數量 (quantity) 的是哪一位會員？請顯示該會員的姓名。"))
time.sleep(30)

INFO | Attempt 1 of 3 in query db data
INFO | Executing SQL: SELECT T1.member_name FROM members AS T1 INNER JOIN transactions AS T2 ON T1.member_id = T2.member_id INNER JOIN transaction_items AS T3 ON T2.transaction_id = T3.transaction_id GROUP BY T1.member_name ORDER BY SUM(T3.quantity) DESC LIMIT 1


{'input': '所有交易紀錄中，單筆交易購買了最多品項數量 (quantity) 的是哪一位會員？請顯示該會員的姓名。', 'output': 'Daniel Ferguson'}
